# Binary Integer Linear Programming (BILP)

### Definition

We are given a vector $c$ of size $N$ and we want to find the vector $x$ of the same size, but with binary values (i.e. $0$ and $1$) such that their dot product $c*x$ is largest. We are also given a constraint for $x$, namely $S*x = b$ for some specified $m \times N$ matrix $S$ and vector $b$ of size $m$.

### Applications

An example real world application of the Binary Integer Linear Programming (BILP) is when a supplier wants to maximize profit, given regulatory constraints.

### Path to solving the problem

BILP is a maximization problem and its cost function can be cast to a QUBO problem through its respective Hamiltonian (see the [Introduction](./introduction_combinatorial_optimization.ipynb) and a [reference](https://arxiv.org/abs/1302.5843)),

$$ \displaystyle \large
H = A \displaystyle \textstyle\sum\limits_{j=1}^{m} \left[ b_j -\textstyle\sum\limits_{i=1}^{N} S_{ji} x_i  \right] ^2 - B \textstyle\sum\limits_{i=1}^{N} c_i x_i
$$

where $i$ and $j$ specify the components of $b$, $S$, $c$ and $x$. $A$ and $B$ are positive constants that need to be chosen such that $A \gg B$. This would ensure that the problem is correctly encoded. Otherwise, the spin configuration for the lowest energy $H$ may not correspond to the best solution of our BILP problem or even to a valid one. 

The QLM allows us to encode a problem in this Hamiltonian form by using the `BILP` class and providing $b$, $S$, $c$ and constants $A$ and $B$. We can then create a job from the problem and send it to a heuristic Simulated Quantum Annealer (SQA) wrapped inside a Quantum Processing Unit (QPU). The SQA will minimize the Hamiltonian, hence find the best solution to our BILP problem.

For a more detailed explanation and a step-by-step guidance, please follow the sections below.

### Quantum resources

To represent the problem as QUBO the QLM would need $N$ spins for encoding each of the binary values in $x$. 

# Example problem

We will construct a problem, which will have more than one $x$ obeying the constraint $S*x = b$. However, we will specify $c$ such that only one $x$ gives a largest value and we will find it! With the procedure to be described, one will be able to solve any instance of a BILP problem.

In [1]:
import numpy as np

# Specify the problem; the solution is x = [0, 1, 1, 0]
c = np.array([0, 1, 1, 1])
S = np.array([[1, 0, 1, 1], [0, 1, 0, 1]])
b = np.array([1, 1])

# Impose constraints for the right encoding
B = 1 
A = 10 * B

Once the problem is specified and the constants $A$ and $B$ properly chosen, we can call our `BILP` class to encode the problem.

In [2]:
from qat.opt import BILP

bilp_problem = BILP(c, S, b, A, B)

# Solution

Now we can proceed to compute the solution by following the steps:

1. Extract the best SQA parameters found for BILP by calling the method `get_best_parameters()`.

    The number of Monte Carlo updates is the total number of updates performed for each temperature (and gamma) on the spins of the equivalent 2D classical system. These updates are the product of the number of annealing steps $-$ `n_steps`, the number of "Trotter replicas" $-$ `n_trotters`, and the problem size, i.e. the number of qubits needed. Hence, we can use these parameters to get the best inferred value for `n_steps`. In general, the more these steps are, the finer and better the annealing will be. However this will cause the process to take longer to complete.
    
    Similarly for the `n_trotters` field in `SQAQPU` $-$ the higher it is, the better the final solution could be, but the more time taken by the annealer to reach an answer.


2. Create a temperature and a gamma schedule for the annealing.

    We use the extracted max and min temperatures and gammas to create a (linear) temperature and a (linear) gamma schedule. These schedules evolve in time from higher to lower values since we simulate the reduction of temperatures and magnetic fields. If one wishes to vary them it may help if the min values are close to $0$, as this will cause the Hamiltonian to reach a lower energy state, potentially closer to its ground state (where the solution is encoded).

    It should be noted that non-linear schedules may be investigated too, but for the same number of steps they could lead to a slower annealing. The best min and max values for gamma and the temperature were found for linear schedules.


3. Generate the SQAQPU and create a job for the problem. The job is then sent to the QPU and the annealing is performed.


4. Present the solution spin configuration and the respective $x$. 


5. Calculate the value of $c*x$.

The solution configuration is a sequence of spins. The position $i$ of each spin in the array corresponds to the position of the $i$-th component of the vector $x$. If a spin has the value $1$ or $-1$, this means that $x_i$ is $1$ or $0$, respectively. 

In [3]:
from qat.sqa import SQAQPU
from qat.core import Variable
from qat.sqa.sqa_qpu import integer_to_spins

# 1. Extract parameters for SQA
problem_parameters_dict = bilp_problem.get_best_parameters()
n_monte_carlo_updates = problem_parameters_dict["n_monte_carlo_updates"]
n_trotters = problem_parameters_dict["n_trotters"]
n_steps = int(n_monte_carlo_updates /
              (n_trotters * len(c))) # the last one is the number of spins, i.e. the problem size
temp_max = problem_parameters_dict["temp_max"]
temp_min = problem_parameters_dict["temp_min"]
gamma_max = problem_parameters_dict["gamma_max"]
gamma_min = problem_parameters_dict["gamma_min"]

# 2. Create a temperature and a gamma schedule
tmax = 1.0
t = Variable("t", float)
temp_t = temp_min * (t / tmax) + temp_max * (1 - t / tmax)
gamma_t = gamma_min * (t / tmax) + gamma_max * (1 - t / tmax)


# 3. Create a job and send it to a QPU
problem_job = bilp_problem.to_job(gamma_t=gamma_t, tmax=tmax, nbshots=1)
sqa_qpu = SQAQPU(temp_t=temp_t, n_steps=n_steps, n_trotters=n_trotters, seed=9999)
problem_result = sqa_qpu.submit(problem_job)

# 4. Present best configuration
state_int = problem_result.raw_data[0].state.int  # raw_data is a list of Samples - one per shot
solution_configuration = integer_to_spins(state_int, len(c))
print("Solution configuration: \n" + str(solution_configuration) + "\n")
x = [1 if spin == 1 else 0 for spin in solution_configuration]
print("x is respectively: \n" + str(x) + "\n")

# 5. Calculate c * x
print("Largest value of c * x:\n" + str(np.dot(c,x)))

Solution configuration: 
[-1.  1.  1. -1.]

x is respectively: 
[0, 1, 1, 0]

Largest value of c * x:
2


# Solution analysis

A simple check of the validity of the answer would be whether $S * x$ actually equals $b$ $-$ the problem constraint that the final result should obey.

In [4]:
print("The constraint S * x = b is " + ("obeyed!" if np.array_equal(np.dot(S, x), b) else "not obeyed."))

The constraint S * x = b is obeyed!


If the constraint is not obeyed, we can determine how much the initial $b$ and the resulting $b$ actually differ, by absolute value.

In [5]:
resulting_b = np.dot(S, x)
difference = b - resulting_b

difference_magnitude = np.linalg.norm(difference)
b_magnitude = np.linalg.norm(b)
fractional_difference = difference_magnitude / b_magnitude

print("The vectors differ by {:.1%}".format(fractional_difference))

The vectors differ by 0.0%
